In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 20)

base = pd.read_csv(
    r"C:\Users\Hooman Deghani\OneDrive\PC\Desktop\Current Datasets\quote activity by page.csv"
)

base.loc[:, 'period'] = pd.to_datetime(base.loc[:, 'period'])

C:\Users\Hooman Deghani\AppData\Local\Temp\ipykernel_13000\2543247844.py:10: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  base.loc[:, 'period'] = pd.to_datetime(base.loc[:, 'period'])


In [2]:
# filter for 2022 only

df = base.loc[('2022-01-01' <= base.loc[:, 'period']) &
 (base.loc[:, 'period'] <= '2022-12-29'),]

# drop unneeded columns
df = df.drop(columns = df.columns.difference([
    'quarter', 'period', 'content_same_visit___num_initiated_quotes',
    'content_same_visit___num_quotes', 'content_same_visit___num_sales',
    'action_page.url'
]))

# add "/" to start of action_page.url to match GA
df.loc[:, 'action_page.url'] = df.loc[:, 'action_page.url'].apply(
    lambda x: "/" + x
)

# perform some sort of sum for each action page to just be unified

jo = df.groupby('action_page.url').sum(numeric_only=True)
jo = jo.sort_values(by='content_same_visit___num_initiated_quotes',
 ascending=False)

jo = jo.reset_index()

In [3]:
# fix up GA's data

ga = pd.read_csv(
    r"C:\Users\Hooman Deghani\OneDrive\PC\Desktop\Current Datasets\Analytics Main (including login data) Channels 20220101-20230101 (1).csv",
    comment = "#"
)

# drop unneeded columns
ga = ga.drop(
    
    columns = ga.columns.difference([
        'Landing Page', 'Sessions'
    ])
)

In [4]:
# merge two dataframes
df = pd.merge(
    left=ga, right=jo, left_on='Landing Page', right_on='action_page.url',
    how='outer', sort=False
)

df = df.dropna(subset=(['action_page.url', 'Sessions']), how='any')

In [5]:
# Turn sessions into float data type
df.loc[:, 'Sessions'] = df.loc[:, 'Sessions'].apply(
    lambda x: x.replace(',', '')
).apply(
    lambda x: float(x)
)

C:\Users\Hooman Deghani\AppData\Local\Temp\ipykernel_13000\2769899595.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'Sessions'] = df.loc[:, 'Sessions'].apply(


In [6]:
# Add new column: conversion rate

df.loc[:, 'Conversion Rate %'] = (
    (df.loc[:, 'content_same_visit___num_sales'] / df.loc[:, 'Sessions']
    * 100).round(decimals=2) 
)

df.shape

(511, 7)

In [7]:
# load and clean up ahrefs data
rt = pd.read_csv(r"C:\Users\Hooman Deghani\OneDrive\PC\Desktop\Current Datasets\rank-tracker.export (3).csv")

# drop da bs
rt = rt.drop(
    columns = rt.columns.difference([
        'URL', 'Position'
    ])
)

rt = rt.dropna(how='any')

# fix the url
rt.loc[:, 'URL'] = rt.loc[:, 'URL'].apply(
    lambda x: x.replace("https://www.squareone.ca", "")
)

# turn numbers into float
rt.loc[:,'Position'] = rt.loc[:, 'Position'].apply(
    lambda x: float(x)
)

rt = rt.groupby('URL').mean().reset_index()

rt

C:\Users\Hooman Deghani\AppData\Local\Temp\ipykernel_13000\3752580715.py:19: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  rt.loc[:,'Position'] = rt.loc[:, 'Position'].apply(


,URL,Position
0,/,13.75
1,/condo,31.00
2,/condo/alberta,7.00
3,/condo/alberta/calgary,4.50
4,/condo/bc,6.50
...,...,...
412,/tenant/quebec,3.60
413,/tenant/saskatchewan/regina,1.00
414,/types-specialty-property,7.00
415,/vacation-property,7.00


In [8]:
# final merge and edits

df = pd.merge(
    left = df, right = rt, left_on = "Landing Page", right_on = "URL",
    how = 'outer', sort = False
)

In [32]:
# final edits to the df
df = df.dropna(subset= ['URL', 'Landing Page'], how= 'any')
df.loc[:, 'Position'] = df.loc[:, 'Position'].apply(
    lambda x: round(x, 2)
)
df = df.drop(columns=['URL', 'action_page.url'])

In [36]:
result = df.loc[
    (df.loc[:, 'Conversion Rate %'] > 5) &
    (df.loc[:, 'Position'] > 3)
]

result.to_csv('results.csv')